In [9]:
import pandas as pd
import os
from itables import show

In [33]:
# Directory containing the CSV files
csv_directory = r"D:\Data Science\VS Code\final_project\Verkehrsdataen_Darmstadt\Verkehrsdaten Darmstadt"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

In [34]:
# Initialize an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Loop through the CSV files and concatenate them
for csv_file in csv_files:
    file_path = os.path.join(csv_directory, csv_file)
    df = pd.read_csv(file_path, sep=';')
    merged_data = pd.concat([merged_data, df], ignore_index=True)

merged_data.reset_index(drop=True, inplace=True)


In [40]:
show(merged_data)

In [ ]:
merged_data.to_csv("/path/to/output/merged_data.csv", index=False)


In [36]:
merged_data['Timestamp'] = pd.to_datetime(merged_data['Datum'] + ' ' + merged_data['Uhrzeit'], format='%d.%m.%Y %H:%M')
#merged_data.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

In [39]:
merged_data['Timestamp'] = pd.to_datetime(merged_data['Timestamp'])
merged_data['roundedTimestamp'] = merged_data['Timestamp'].dt.round('30min')

In [46]:
show(merged_data.loc[merged_data['roundedTimestamp'] == '2022-08-26 07:00:00'], maxBytes=0)

In [22]:
columns_to_sum = merged_data.columns.difference(['Bezeichnung', 'Timestamp'])
grouped_data = merged_data.groupby(['Bezeichnung', 'Timestamp'])[columns_to_sum].sum().reset_index()
show(grouped_data)